In [ ]:
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

def sort_contours(cnts,reverse = False):
    i = 0
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
                                        key=lambda b: b[1][i], reverse=reverse))
    return cnts

def predict_from_model_128(image,model,labels):
    image = cv2.resize(image,(128,128))
    image = np.stack((image,)*3, axis=-1)
    prediction = labels.inverse_transform([np.argmax(model.predict(image[np.newaxis,:]))])
    return prediction

def predict_from_model_80(image,model,labels):
    image = cv2.resize(image,(80,80))
    image = np.stack((image,)*3, axis=-1)
    prediction = labels.inverse_transform([np.argmax(model.predict(image[np.newaxis,:]))])
    return prediction

def minimum_character(input_list):

    if len(input_list) == 0:
        return -1
    A = np.array(sorted(input_list))
    #print(A)
    z = np.abs(stats.zscore(A))
    A_clean = np.array([A[i] for i in range(len(z)) if z[i] < 2])
    return A_clean[0]


def recognize_char(input_img, first_letter_model,first_letter_labels, second_letter_model, second_letter_labels,digit_model, digit_labels, model, labels):
    blur = cv2.bilateralFilter(input_img,9,95,95)
    blur = cv2.detailEnhance(blur, 5, 0.95)
    gray_img = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)
    canny_edges = cv2.Canny(gray_img,220, 250, None, 5)
    lines = cv2.HoughLines(canny_edges, 1, np.pi / 180, 50, None, 0, 0)
    avg_theta = 0
    theta =0
    line_count = 0
    if lines is not None:
        for i in range(0, len(lines)):
            theta = lines[i][0][1]
            angle = (180*theta/3.1415926 - 90)
            if -15<=angle<=15:
                avg_theta += angle
                line_count += 1
    if line_count != 0:
        avg_theta = avg_theta/line_count
        img_rotated = rotate_image(input_img, avg_theta)
    else:
        img_rotated = rotate_image(input_img, 0)
    enhanced_img = cv2.detailEnhance(img_rotated, 9, 10, 0.5)
    enhanced_gray_img = cv2.cvtColor(enhanced_img, cv2.COLOR_BGR2GRAY)
    blur_enhanced_gray_img = cv2.bilateralFilter(enhanced_gray_img, 9,10,10)
    binary_image = cv2.threshold(blur_enhanced_gray_img, 100,255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    cont, _  = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    char_segmented = []
    char_w = 96
    char_h = 96
    char_ratio_list = []
    for c in sort_contours(cont):
        (x, y, w, h) = cv2.boundingRect(c)
        ratio = h/w
        if 1<=ratio<=5: # Only select contour with defined ratio
            char_ratio_list.append(h/img_rotated.shape[0])
    mini_char = minimum_character(char_ratio_list)
    print("Testing code");
    print(mini_char)
    if mini_char == -1:
        flag = False
    else:
        flag = True
    if(flag):
        for c in sort_contours(cont):
            (x, y, w, h) = cv2.boundingRect(c)
            ratio = h/w
            if 1<=ratio<=5:
                if 0.3 <= h/img_rotated.shape[0] <= 0.9:
                    curr_num = binary_image[y:y+h,x:x+w]
                    curr_num = cv2.resize(curr_num, dsize=(char_w, char_h))
                    _, curr_num = cv2.threshold(curr_num, 90, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                    char_segmented.append(curr_num)
        final_string = ''
        title = np.array2string(predict_from_model_128(char_segmented[0],first_letter_model,first_letter_labels))
        final_string +=title.strip("'[]")
        title = np.array2string(predict_from_model_128(char_segmented[1],second_letter_model,second_letter_labels))
        final_string +=title.strip("'[]")
        title = np.array2string(predict_from_model_128(char_segmented[2],digit_model,digit_labels))
        final_string +=title.strip("'[]")
        title = np.array2string(predict_from_model_128(char_segmented[3],digit_model,digit_labels))
        final_string +=title.strip("'[]")
        for i in range(4,len(char_segmented)-4):
            title = np.array2string(predict_from_model_80(char_segmented[i],model,labels))
            final_string+=title.strip("'[]")
        for i in range(len(char_segmented)-4,len(char_segmented)):
            title = np.array2string(predict_from_model_128(char_segmented[i],digit_model,digit_labels))
            final_string+=title.strip("'[]")
        return final_string
    else:
        return pytesseract.image_to_string(blur_enhanced_gray_img)